# Importing necessary libraries

In [ ]:
import cv2
import numpy as np
import glob
from matplotlib import pyplot as plt

In [ ]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator


In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D
from tensorflow.keras.layers import Flatten, Dense, Dropout, AveragePooling2D
from tensorflow.keras.optimizers import Adam

In [ ]:
from keras.applications import VGG19
from keras.models import Model
import keras_tuner
from keras_tuner.tuners import RandomSearch

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
from sklearn.svm import SVC

In [ ]:
from sklearn.metrics import classification_report, accuracy_score

# Dataset reading and display

In [ ]:
glaucoma_images = [cv2.imread(file, cv2.IMREAD_COLOR) for file in glob.glob("/kaggle/input/kcgdataset/fundus-images-glaucoma-normal/*.PNG")]
normal_images = [cv2.imread(file, cv2.IMREAD_COLOR) for file in glob.glob("/kaggle/input/kcgdataset/fundus-images-glaucoma-normal/*.jpg")]
# df = pd.read_csv("./Drishti/Training/Images/")

In [ ]:
num_glaucoma = len(glaucoma_images)
num_normal = len(normal_images)
print(num_glaucoma)
print(num_normal)

As the data is very less and not balanced this would cause overfitting as well as bias in the classification

# Data Augmentation

In [ ]:
difference = abs(num_glaucoma - num_normal)

In [ ]:
if num_glaucoma > num_normal:
    category_to_augment = "normal"
    num_original_images = num_normal
else:
    category_to_augment = "glaucoma"
    num_original_images = num_glaucoma

In [ ]:
# Calculate the number of augmented images needed per original image
augments_per_image = -(-difference // num_original_images)  # This is a way to do a ceiling division

In [ ]:
# Data augmentation configuration
datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=40,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

In [ ]:
def augment_images(image_list, augments_per_image):
    augmented_images = []
    
    for img in image_list:
        img = img.reshape((1,) + img.shape)
        aug_iter = 0
        for aug_img in datagen.flow(img, batch_size=1):
            augmented_images.append(aug_img[0])
            aug_iter += 1
            if aug_iter >= augments_per_image:
                break
                
    return augmented_images

In [ ]:
def resize_image(image, target_size=(224, 224)):
    return cv2.resize(image, target_size)

# First, resize the images
glaucoma_images_resized = [resize_image(img) for img in glaucoma_images]
normal_images_resized = [resize_image(img) for img in normal_images]

In [ ]:
if category_to_augment == "normal":
    augmented_normal_images = augment_images(normal_images_resized, augments_per_image)
    normal_images_resized.extend(augmented_normal_images)
else:
    augmented_glaucoma_images = augment_images(glaucoma_images_resized, augments_per_image)
    glaucoma_images_resized.extend(augmented_glaucoma_images)

In [ ]:
countga = len(glaucoma_images_resized)
countna = len(normal_images_resized)
print(countga)
print(countna)

In [ ]:
def display_images(image_list, title_list, columns=5):
    n = len(image_list)
    rows = n // columns
    rows += n % columns

    position = range(1, n + 1)

    plt.figure(figsize=(20, 20))

    for k, (img, title) in enumerate(zip(image_list, title_list)):
        plt.subplot(rows, columns, position[k])
        plt.imshow(img, cmap='gray')
        plt.title(title)
        plt.axis('on')

    plt.show()

# Titles for images
glaucoma_titles = ['Glaucoma' for _ in glaucoma_images_resized]
normal_titles = ['Normal' for _ in normal_images_resized]

# Display glaucoma images
display_images(glaucoma_images_resized, glaucoma_titles)

# Display normal images
display_images(normal_images_resized, normal_titles)

In [ ]:
# Green Channel Extraction:
def extract_green_channel(img):
    # OpenCV reads images in the format BGR. The second channel (index 1) is the green channel.
    return img[:, :, 1]

glaucoma_green = [extract_green_channel(img) for img in glaucoma_images_resized]
normal_green = [extract_green_channel(img) for img in normal_images_resized]


In [ ]:
# Just for visualization
def plot_before_after(original, green_channel, title):
    fig, ax = plt.subplots(1, 2, figsize=(12, 6))
    
    ax[0].imshow(cv2.cvtColor(original, cv2.COLOR_BGR2RGB))
    ax[0].set_title('Original Image')
    ax[0].axis('off')

    ax[1].imshow(green_channel, cmap='gray')
    ax[1].set_title('Green Channel')
    ax[1].axis('off')
    
    plt.suptitle(title)
    plt.show()

# Displaying the first image for demonstration
plot_before_after(glaucoma_images_resized[0], glaucoma_green[0], 'Glaucoma Image')
plot_before_after(normal_images_resized[0], normal_green[0], 'Normal Image')

In [ ]:
labels_glaucoma = np.ones(len(glaucoma_green))
labels_normal = np.zeros(len(normal_green))

In [ ]:
X = glaucoma_green + normal_green
y = [1]*len(glaucoma_green) + [0]*len(normal_green)  # Assuming 1 for glaucoma and 0 for normal

In [ ]:
unique_shapes = {img.shape for img in X}
print(unique_shapes)

In [ ]:
desired_shape = (224, 224)
X_resized = [cv2.resize(img, desired_shape) for img in X]

In [ ]:
X_green = np.array(X_resized, dtype="float32") / 255.0
X_green = X_green.reshape(X_green.shape[0], X_green.shape[1], X_green.shape[2], 1)  # Add a channel dimension
# Convert y to numpy array
y_green = np.array(y)  

#y_green = y_green.reshape(y_green.shape[0], 1)# Labels remain the same
print("X_green shape:", X_green.shape)
print("y_green shape:", y_green.shape)


In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X_green, y_green, test_size=0.2, stratify=y_green, random_state=42)

CNN

In [ ]:
from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, UpSampling2D, concatenate, Flatten, Dense

def build_unet(input_shape):
    inputs = Input(input_shape)

    # Encoder
    conv1 = Conv2D(64, (3, 3), activation='relu', padding='same')(inputs)
    pool1 = MaxPooling2D(pool_size=(2, 2))(conv1)

    # Bottom layer
    conv2 = Conv2D(128, (3, 3), activation='relu', padding='same')(pool1)

    # Decoder
    up1 = concatenate([UpSampling2D(size=(2, 2))(conv2), conv1], axis=-1)
    conv3 = Conv2D(64, (3, 3), activation='relu', padding='same')(up1)

    # Flatten layer
    flatten = Flatten()(conv3)

    # Dense layers for classification
    dense1 = Dense(64, activation='relu')(flatten)
    outputs = Dense(1, activation='sigmoid')(dense1)

    # Create model
    model = Model(inputs=inputs, outputs=outputs)
    return model

# Assuming X_train is the dataset after preprocessing
# And the images in X_train have been reshaped to have a channel dimension
input_shape = X_train[0].shape

# Build the model
model = build_unet(input_shape)

# Compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Display model summary
model.summary()


In [ ]:
# Train the model
history = model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=10, batch_size=32)


In [ ]:
loss_CNN, accuracy_CNN = model.evaluate(X_test, y_test)
print(f"Validation Loss for CNN: {loss_CNN:.4f}")
print(f"Validation Accuracy for CNN: {accuracy_CNN:.4f}")

ROC curve

In [ ]:
from sklearn.metrics import roc_curve, auc

train_predictions = model.predict(X_train)  
test_predictions = model.predict(X_test)   

# Calculate the ROC curve and AUC for the training data
fpr_train, tpr_train, _ = roc_curve(y_train, train_predictions)
roc_auc_train = auc(fpr_train, tpr_train)

# Calculate the ROC curve and AUC for the testing data
fpr_test, tpr_test, _ = roc_curve(y_test, test_predictions)
roc_auc_test = auc(fpr_test, tpr_test)

# Plot the ROC curves
plt.figure(figsize=(8, 6))
plt.plot(fpr_train, tpr_train, color='darkorange', lw=2, label=f'Training ROC curve (area = {roc_auc_train:.2f})')
plt.plot(fpr_test, tpr_test, color='cornflowerblue', lw=2, label=f'Testing ROC curve (area = {roc_auc_test:.2f})')
plt.plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('KCG')
plt.legend(loc='lower right')
plt.show()

In [ ]:
y_pred_binary = (test_predictions > 0.5).astype(int)

Confusion Matrix

In [ ]:
from sklearn.metrics import confusion_matrix, classification_report

conf_matrix = confusion_matrix(y_test, y_pred_binary)

# True Positive, True Negative, False Positive, False Negative
tp = conf_matrix[1, 1]
tn = conf_matrix[0, 0]
fp = conf_matrix[0, 1]
fn = conf_matrix[1, 0]

In [ ]:
sensitivity = tp / (tp + fn)
specificity = tn / (tn + fp)
precision = tp / (tp + fp)
recall = sensitivity

# F1 Score
f1_score = 2 * (precision * recall) / (precision + recall)

# Print the metrics
print(f"Sensitivity: {sensitivity:.4f}")
print(f"Specificity: {specificity:.4f}")
print(f"Precision: {precision:.4f}")
print(f"Recall: {recall:.4f}")
print(f"F1 Score: {f1_score:.4f}")

Classification Report

In [ ]:
print("\nClassification Report:")
print(classification_report(y_test, y_pred_binary))